In [ ]:
API_KEY = 'AIzaSyC3Fgt_Cx1PpWe8DEUA5sjpaIiWlTHOJN' 
# Q
# gemini-2.5-flash-preview-05-20

In [ ]:
# from langchain_core.messages import HumanMessage
# from langchain_google_genai import ChatGoogleGenerativeAI
# import PIL.Image

# llm = ChatGoogleGenerativeAI(api_key=API_KEY,model="gemini-2.5-flash-preview-05-20")
# # example
# image = PIL.Image.open(r'C:\VScodeMaster\Inferencing\Extractor\data\reference-architecture.png')

# hmessage1 = HumanMessage(
#     content=[
#         {
#             "type": "text",
#             "text": "Extract all the component of AWS from image and inlist them in a json?",
#         },  # You can optionally provide text parts
#         {"type": "image_url", "image_url": image},
#     ]
# )
# message1 = llm.invoke([hmessage1])
# print(message1.content)

In [5]:
from langchain_core.messages import HumanMessage,SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_ollama.chat_models import ChatOllama
import PIL.Image
import base64
llm = ChatGoogleGenerativeAI(api_key=API_KEY,model="gemini-2.5-flash-preview-05-20")
from pydantic import BaseModel
# llm = ChatOllama(model='gemma3:4b')
# example
image_path = r'C:\VScodeMaster\Inferencing\Extractor\data\reference-architecture.png'
# image = PIL.Image.open()
# image_data = base64.b64encode(r'C:\VScodeMaster\Inferencing\Extractor\data\reference-architecture.png')

with open(image_path, "rb") as image_file:
    encoded_image = base64.b64encode(image_file.read()).decode("utf-8")
systemm = SystemMessage(
    content="""You are an Amazon Web Services expert having expertise in interpertation on architectural diagrams and extraction of all the components and identification of all the components"""
)
hmessage1 = HumanMessage(
    content=[
        {
            "type": "text",
            "text": "Extract all the component of AWS from image make sure all the services are there in image and list them in json and also there relationship with other how they are in same json with 'relation' as key  ",
        },  # You can optionally provide text parts
        {
            "type": "image",
            "source_type": "base64",
            "data": encoded_image,
            "mime_type": "image/png",
        },
    ]
)
from typing import List,Literal
class relation(BaseModel):
    target : str
    description: str 
class Services(BaseModel):
    name : str
    type : Literal['AWS service' , 'other']
    description : str
    account_context : str
    relations : List[relation]
class Group(BaseModel):
    name: str
    services : List[Services]
class Diagram(BaseModel):
    Groups : List[Group]
# class Services(BaseModel):
#     services : List[str]
st = llm.with_structured_output(Diagram)
# message1 = llm.invoke([hmessage1])
# print(message1.content)

print("-----------------------------------------------------------------------------------")
message1 = st.invoke([systemm,hmessage1])
print(message1)
# message2 = llm2.invoke([hmessage1])
# print(message2.content)

-----------------------------------------------------------------------------------
Groups=[Group(name='Pipeline provisioning', services=[Services(name='Amazon S3 (Model artifact and data)', type='AWS service', description='Model artifact and data', account_context='AWS Cloud', relations=[relation(target='Orchestrator', description='Receives model artifacts and training data from')]), Services(name='Amazon SageMaker Model Registry', type='AWS service', description='Model Registry', account_context='AWS Cloud', relations=[]), Services(name='Amazon ECR', type='AWS service', description='Custom algorithms', account_context='AWS Cloud', relations=[]), Services(name='Amazon API Gateway (Pipeline provisioning)', type='AWS service', description='Option 1: Initiate by API call', account_context='AWS Cloud', relations=[relation(target='Orchestrator', description='Receives API calls from'), relation(target='AWS Lambda (Pipeline provisioning)', description='Invokes')]), Services(name='AWS Lambda 

In [ ]:
# print(message1)
import json 
for group in message1.Groups:
    print(json.dumps(group.model_dump(),indent=2))

In [ ]:
[{
  "name": "Pipeline provisioning",
  "services": [
    {
      "name": "Amazon S3",
      "type": "AWS service",
      "description": "Model artifact and data",
      "account_context": "AWS Cloud",
      "relations": [
        {
          "target": "Orchestrator",
          "description": "Model artifacts and training data for model monitor"
        }
      ]
    },
    {
      "name": "Amazon SageMaker",
      "type": "AWS service",
      "description": "Model Registry",
      "account_context": "AWS Cloud",
      "relations": []
    },
    {
      "name": "Amazon ECR",
      "type": "AWS service",
      "description": "Custom algorithms",
      "account_context": "AWS Cloud",
      "relations": []
    },
    {
      "name": "Amazon API Gateway",
      "type": "AWS service",
      "description": "Option 1: Initiate by API call",
      "account_context": "AWS Cloud",
      "relations": [
        {
          "target": "AWS Lambda",
          "description": "Initiate by API call"
        },
        {
          "target": "Orchestrator",
          "description": "Initiate by API call"
        }
      ]
    },
    {
      "name": "Amazon S3",
      "type": "AWS service",
      "description": "Pipeline blueprint repository",
      "account_context": "AWS Cloud",
      "relations": [
        {
          "target": "Orchestrator",
          "description": "Interacts with"
        },
        {
          "target": "AWS Lambda",
          "description": "Triggers"
        }
      ]
    },
    {
      "name": "Amazon S3",
      "type": "AWS service",
      "description": "Option 2: Initiate by upload to S3 bucket",
      "account_context": "AWS Cloud",
      "relations": [
        {
          "target": "AWS Lambda",
          "description": "Initiate by upload to S3 bucket"
        }
      ]
    },
    {
      "name": "AWS Lambda",
      "type": "AWS service",
      "description": "",
      "account_context": "AWS Cloud",
      "relations": [
        {
          "target": "Amazon API Gateway",
          "description": "Triggered by"
        },
        {
          "target": "Amazon S3 (Pipeline blueprint repository)",
          "description": "Triggered by"
        },
        {
          "target": "Amazon S3 (Option 2: Initiate by upload to S3 bucket)",
          "description": "Triggered by"
        },
        {
          "target": "AWS CloudFormation",
          "description": "Triggers"
        }
      ]
    },
    {
      "name": "AWS CloudFormation",
      "type": "AWS service",
      "description": "",
      "account_context": "AWS Cloud",
      "relations": [
        {
          "target": "AWS Lambda",
          "description": "Triggered by"
        },
        {
          "target": "AWS CodePipeline",
          "description": "Deploys"
        }
      ]
    }
  ]
},
{
  "name": "AWS CodePipeline",
  "services": [
    {
      "name": "Amazon S3",
      "type": "AWS service",
      "description": "Stage: Source",
      "account_context": "AWS Cloud",
      "relations": [
        {
          "target": "AWS CloudFormation action",
          "description": "Feeds into"
        }
      ]
    },
    {
      "name": "AWS CloudFormation action",
      "type": "AWS service",
      "description": "Stage: DeployPipeline",
      "account_context": "AWS Cloud",
      "relations": [
        {
          "target": "Amazon S3 (Stage: Source)",
          "description": "Receives input from"
        },
        {
          "target": "Provisioned Pipeline",
          "description": "Deploys"
        }
      ]
    }
  ]
},
{
  "name": "Provisioned Pipeline (e.g. bring your own model - realtime inference pipeline)",
  "services": [
    {
      "name": "Model",
      "type": "other",
      "description": "",
      "account_context": "AWS Cloud",
      "relations": [
        {
          "target": "Amazon SageMaker Endpoint",
          "description": "Associated with"
        }
      ]
    },
    {
      "name": "Amazon SageMaker Endpoint",
      "type": "AWS service",
      "description": "Invoke endpoint",
      "account_context": "AWS Cloud",
      "relations": [
        {
          "target": "Model",
          "description": "Serves"
        },
        {
          "target": "AWS Lambda (Invoke endpoint)",
          "description": "Invoked by"
        }
      ]
    },
    {
      "name": "AWS Lambda",
      "type": "AWS service",
      "description": "Invoke endpoint",
      "account_context": "AWS Cloud",
      "relations": [
        {
          "target": "Amazon SageMaker Endpoint",
          "description": "Invokes"
        },
        {
          "target": "Amazon API Gateway",
          "description": "Triggered by"
        }
      ]
    },
    {
      "name": "Amazon API Gateway",
      "type": "AWS service",
      "description": "",
      "account_context": "AWS Cloud",
      "relations": [
        {
          "target": "User",
          "description": "Interacted with by"
        },
        {
          "target": "AWS Lambda (Invoke endpoint)",
          "description": "Triggers"
        }
      ]
    },
    {
      "name": "Amazon S3",
      "type": "AWS service",
      "description": "Existing bucket used to store captured data, input/output of batch transform jobs, or model monitor",
      "account_context": "AWS Cloud",
      "relations": []
    }
  ]
}]

In [ ]:
# # print(len(message1.services))
# import json
# for service in message1. :
#     print()
#     if service.type == 'AWS service':
#         print(json.dumps(service.model_dump(),indent=2))

#     # print(json.dumps(message1.model_dump(), indent=2))

In [1]:
from langchain_core.messages import HumanMessage,SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_ollama.chat_models import ChatOllama
import PIL.Image
import base64
# llm = ChatGoogleGenerativeAI(api_key=API_KEY,model="gemini-2.5-flash-preview-05-20")
from pydantic import BaseModel
llm = ChatOllama(model='gemma3:4b-it-q4_K_M',keep_alive='2h')
# example
image_path = r'C:\VScodeMaster\Inferencing\Extractor\data\reference-architecture.png'
# image = PIL.Image.open()
# image_data = base64.b64encode(r'C:\VScodeMaster\Inferencing\Extractor\data\reference-architecture.png')

with open(image_path, "rb") as image_file:
    encoded_image = base64.b64encode(image_file.read()).decode("utf-8")
systemm = SystemMessage(
    content="""You are an Amazon Web Services expert having expertise in interpertation on architectural diagrams and extraction of all the components and identification of all the components"""
)
hmessage1 = HumanMessage(
    content=[
        {
            "type": "text",
            "text": "Extract all the component of AWS from image make sure all the services are there in image and list them in json and also there relationship with other how they are in same json with 'relation' as key  ",
        },  # You can optionally provide text parts
        {
            "type": "image",
            "source_type": "base64",
            "data": encoded_image,
            "mime_type": "image/png",
        },
    ]
)
from typing import List,Literal
class relation(BaseModel):
    target : str
    description: str 
class Services(BaseModel):
    name : str
    type : Literal['AWS service' , 'other']
    description : str
    account_context : str
    relations : List[relation]
class Group(BaseModel):
    name: str
    services : List[Services]
class Diagram(BaseModel):
    Groups : List[Group]
# class Services(BaseModel):
#     services : List[str]
st = llm.with_structured_output(Diagram)
# message1 = llm.invoke([hmessage1])
# print(message1.content)

print("-----------------------------------------------------------------------------------")
message1 = st.invoke([systemm,hmessage1])
print(message1)
# message2 = llm2.invoke([hmessage1])
# print(message2.content)


-----------------------------------------------------------------------------------
Groups=[Group(name='Model Training & Deployment Pipeline', services=[Services(name='Amazon S3', type='AWS service', description='Storage service for model artifacts, training data, and batch transforms.', account_context='Primary storage for all pipeline components.', relations=[relation(target='Amazon SageMaker', description='Provides data for training and model deployment.'), relation(target='Amazon S3', description='Used for storing input data and transformed data for batch jobs.'), relation(target='Amazon S3', description='Used for outputting a model after training')]), Services(name='Amazon SageMaker', type='AWS service', description='Managed machine learning service for training and deploying models.', account_context='Central for model development and deployment.', relations=[relation(target='Amazon S3', description='Provides data for training and model deployment.'), relation(target='Amazon S3',

In [2]:
import json 
for group in message1.Groups:
    print(json.dumps(group.model_dump(),indent=2))

{
  "name": "Model Training & Deployment Pipeline",
  "services": [
    {
      "name": "Amazon S3",
      "type": "AWS service",
      "description": "Storage service for model artifacts, training data, and batch transforms.",
      "account_context": "Primary storage for all pipeline components.",
      "relations": [
        {
          "target": "Amazon SageMaker",
          "description": "Provides data for training and model deployment."
        },
        {
          "target": "Amazon S3",
          "description": "Used for storing input data and transformed data for batch jobs."
        },
        {
          "target": "Amazon S3",
          "description": "Used for outputting a model after training"
        }
      ]
    },
    {
      "name": "Amazon SageMaker",
      "type": "AWS service",
      "description": "Managed machine learning service for training and deploying models.",
      "account_context": "Central for model development and deployment.",
      "relations": [
 

In [ ]:
[{
  "name": "Model Training & Deployment Pipeline",
  "services": [
    {
      "name": "Amazon S3",
      "type": "AWS service",
      "description": "Storage service for model artifacts, training data, and batch transforms.",
      "account_context": "Primary storage for all pipeline components.",
      "relations": [
        {
          "target": "Amazon SageMaker",
          "description": "Provides data for training and model deployment."
        },
        {
          "target": "Amazon S3",
          "description": "Used for storing input data and transformed data for batch jobs."
        },
        {
          "target": "Amazon S3",
          "description": "Used for outputting a model after training"
        }
      ]
    },
    {
      "name": "Amazon SageMaker",
      "type": "AWS service",
      "description": "Managed machine learning service for training and deploying models.",
      "account_context": "Central for model development and deployment.",
      "relations": [
        {
          "target": "Amazon S3",
          "description": "Provides data for training and model deployment."
        },
        {
          "target": "Amazon S3",
          "description": "Used for storing output models after training"
        },
        {
          "target": "Amazon S3",
          "description": "Used for storing output models after training"
        },
        {
          "target": "Amazon S3",
          "description": "Used for storing output models after training"
        }
      ]
    },
    {
      "name": "Amazon ECR",
      "type": "AWS service",
      "description": "Container registry for storing custom algorithms",
      "account_context": "Used for storing container images that contain model algorithms.",
      "relations": [
        {
          "target": "Amazon SageMaker",
          "description": "Used to host and deploy custom algorithms as containers."
        }
      ]
    },
    {
      "name": "AWS Lambda",
      "type": "AWS service",
      "description": "Serverless compute service used for various orchestration and transformation tasks.",
      "account_context": "Used for handling serverless functions in the pipeline.",
      "relations": [
        {
          "target": "Amazon S3",
          "description": "Triggered based on events in S3"
        },
        {
          "target": "Amazon S3",
          "description": "Used for processing batch transform jobs"
        }
      ]
    },
    {
      "name": "Amazon API Gateway",
      "type": "AWS service",
      "description": "Creates, publishes, maintains, monitors, and secures APIs.",
      "account_context": "Exposes the deployed model as an API endpoint.",
      "relations": [
        {
          "target": "Amazon S3",
          "description": "Used for handling requests from end users to access the model API."
        },
        {
          "target": "Amazon S3",
          "description": "Used for handling requests from end users to access the model API."
        }
      ]
    },
    {
      "name": "Orchestrator",
      "type": "AWS service",
      "description": "Used to manage and orchestrate the data flow.",
      "account_context": "Used for managing data flow for monitoring.",
      "relations": [
        {
          "target": "Amazon S3",
          "description": "Triggered based on events in S3"
        },
        {
          "target": "Amazon S3",
          "description": "Triggered based on events in S3"
        }
      ]
    }
  ]
}]